# Un modello matematico del testo in linguaggio naturale (naturale=umano)

Questo modello ci aiuterà a capire come funzionano i **LLM** (Large Language Model).

* I LLM sono le retu neurali alla base dell'IA generativa.
(Adesso stanno studiando i LWM, large word model, quindi in grado di muoversi nel mondo, non solo di parlare perché il mondo non e fatto solo di testo, altri modelli che sanno elaborare testo in alcuni casi riescono ad interfacciarsi con strumenti come applicazioni o siti. Esistono poi agenti che girano sul terminale e che possono interfacciarsi direttamente con il computer, caricare su gitub, installare...).

* I LLM generano il testo mediante un preocesso detto **autoregressione**: quando forniamo un prompt all'IA generativa, il LLM genera la risposta **token** per **token**. un token è l'unità in cui un LLM divide il testo in input. Poi ogni token viene mandato ad un enorme dizionario in py che ha un numero associato, il LLM riceve una serie di numeri. In media un token è 0,75 parole, (Tiktokenizer). QUANDO GENERA inizialmente ha come input sono il prompt, dopo aver generato il primo token il LLM ha come input il token precedente e il prompt, dal secondo in avanti genera grazie a tutti i token precedenti più il prompt. Dalla seconda domanda in avanti l'input saranno tutti i token del primo prompt, tutti i token da lui generati e tutti i token del secondo prompt. Mano a mano che l'input aumenta il LLM rallenta. L'insieme dei tokek dei prompt e quelli generati si chiama **contesto** *A ogni istante della generazione della risposta, LLM prendi in inpput tutti i token del prompt e tutti i nuovi token che generano fino a quel momento.* I LLM non sono davvero intelligenti, noi umani pensiamo prima e scriviamo in base al pensiero e non a quello scritto fino a quel punto.


*scalabile: un sw in informatica è scalabile quando è in grado di funzionare indipendentemente dal numero di client che ci vogliono accedere*.

## Differenza tra LLM e autocompletamento della tastiera 

nelle reti neurali umane quando pensiamo avvengono dei cambiamenti fisici tra sinapsi, ovvero collegamenti tra neuroni, nel momento in cui pensiamo alcuni collegamenti si attenuano altri si rinforzano. Nelle reti neurali anche i percettroni hanno collegamenti con altri percettroni e ha dei paramentri, ovvero numeri, associati che si modificano in base alla forza che devono avere con altri percettroni. Il problema dei LLM è che anno circa 10^13 parametri. Il perchè il deep learning funziona così bene non è nonto ancora. 

L'autocompletamento di solito non hanno reti neurali.

In [48]:
import random

### caricamento input con formato utf 8


In [17]:
file=open("./data/sei_personaggi_in_cerca_dautore_Luigi_Pirandello.txt", "r", encoding="utf-8")
text=file.read()
#print(text)

In [3]:
text.replace?

Signature: text.replace(old, new, count=-1, /)
Docstring:
Return a copy with all occurrences of substring old replaced by new.

  count
    Maximum number of occurrences to replace.
    -1 (the default value) means replace all occurrences.

If the optional argument count is given, only the first count occurrences are
replaced.
Type:      builtin_function_or_method

### Pre-processing

In [39]:
text=text.replace("\n", " ").lower() # abbiamo tolto le messe a capo sostituendole con spazio. poi resto tutto minuscolo
for _ in range(2):
    text=text.replace("  ", " ")

text

"sei personaggi in cerca d'autore di luigi pirandello i personaggi della commedia da fare il padre la madre la figliastra il figlio il giovinetto la bambina (questi ultimi due non parlano) (poi, evocata) madama pace gli attori della compagnia il direttore-capocomico la prima attrice il primo attore la seconda donna l'attrice giovane l'attor giovane altri attori e attrici il direttore di scena il suggeritore il trovarobe il macchinista il segretario del capocomico l'uscere del teatro apparatori e servi di scena di giorno, su un palcoscenico di teatro di prosa. n.b. la commedia non ha atti né scene. la rappresentazione sarà interrotta una prima volta, senza che il sipario s'abbassi; allorché il direttore capocomico e il capo dei personaggi si ritireranno per concertar lo scenario e gli attori sgombreranno il palcoscenico; una seconda volta, allorché per isbaglio il macchinista butterà giù il sipario. troveranno gli spettatori, entrando nella sala del teatro, alzato il sipario, e il palco

In [84]:
# i nostri token sono sempre composti da un carattere
l_context=15  #un LLM tiene conto di 200 mila token, noi teniamo conto di due token dove ogni token per noi è solo una lettera. 

markov_dict={} # che contiente chiave lettere*l_context e valore lista dei successivi. 
for i, _ in enumerate(text[:-l_context]): #ciclando sul testo con indice carattere e carattere. Non deve arrivare in fondo perchè accederò alle posizioni successive all'attuale
    ngram=text[i:i+l_context]#slicing per avere il numero di lettere consecutive
    if ngram in markov_dict:
        markov_dict[ngram].append(text[i+l_context])# vado a salvarmi la lettera succesiva all'ennegramma che sto analizzando
    else:
        markov_dict[ngram]=[text[i+l_context]]


In [68]:
markov_dict["sei personaggi"]

[' ', ',', '»', ',', ' ', '.']

il dizionario `markov_dict` è un modello matematico del linguaggio naturale che ora usiamo per costruire un autoregressione markoviana.

In [99]:
ngram_0="sei personaggi»"
l_phrase=600
phrase=ngram_0# in phrase accumulo, creo tutta la frase

if len(ngram_0)!=l_context:
    print("Errore!")
else:
    for _ in range(l_phrase):
        if ngram_0 in markov_dict:
            next_char=random.choice(markov_dict[ngram_0])
            phrase=phrase+next_char
            ngram_0=phrase[-l_context:]
        else: 
            break

phrase

"sei personaggi» non dovranno infatti apparire come «fantasmi», ma come «realtà create», costruzioni della fantasia immutabili: e dunque più reali e consistenti della volubile naturalità degli attori. il capocomico (alla prima attrice il cenno di sì. «c'è venuta qualche altra volta? più d'una?» il capocomico. e com'è dunque vedova, se lei è vivo? gli attori scaricheranno tutto il loro sbalordimento in una fragorosa risata. il padre (ferito). mi dispiace che ridano così, perché portiamo in noi, ripeto, un dramma doloroso, come lor signori possono argomentare da questa donna velata di nero. così dicendo porgerà"

## compito 
fare una classe che implementa un autogerenazione markoviana